### Выгрузка исторических данных по списку сайтов и разным сегментам
Задача была в быстрой выгрузке исторических данных для нескольких сайтов по всему трафику, региональному трафику, трафику за исключением внутреннего трафика дзен сообществ (всему и региональному). Необходимые для выгрузки данные: стандартные метрики посещаемости и вовлеченности, а также объем трафика сайтов/сегментов в структуре и по отдельным каналам. 
Задача разбита на 4 функции, где добавлялись/исключались фильтры для Метрики для необходимого сегмента (неколько этапов).
Скрипт частично отредактирован, так как содержит данные заказчика.

In [ ]:
import pandas as pd
import json
import requests
import re

In [ ]:
# Задаю Токен и параметры для API запроса
header = {'Authorization': #TOKEN }

ids = [
         {'name': '', 'id': '', 'region': 'RU-NVS'},
         {'name': '', 'id': '', 'region': 'RU-CHE'},
         {'name': '', 'id': '', 'region': 'RU-KK'},
         {'name': '', 'id': '', 'region': 'RU-KHM'},
      ]

In [ ]:
# 1 запрос. Показатели посещаемости и вовлечения
# Функция возвращает ДАТАФРЕЙМ с метриками Пользователи, Визиты, Просмоты, Глубина, Время, Отказы

def query1(counter, region):
    exclude_zen = f"ym:pv:URLDomain!@'zen.yandex' AND ym:s:startURL=@'{counter[1]}'"
    region_fltr = f"ym:s:regionAreaIsoName=='{region}'"
    payload = {
        'metrics': ["ym:s:users","ym:s:visits","ym:s:pageviews","ym:s:pageDepth", "ym:s:avgVisitDurationSeconds","ym:s:bounceRate"],
    #     'metrics': 'ym:s:visits',
        'accuracy': '1',
        'attribution': 'last',
        'date1': '2015-12-01',
        'date2': '2021-03-31',
        'group': 'month',
        'filters': f"{region_fltr} AND {exclude_zen}",
#         'filters': f"ym:s:regionAreaIsoName=='{region}'",
        'ids': counter[0]
        }

    r = requests.get('https://api-metrika.yandex.ru/stat/v1/data/bytime', params=payload, headers=header)
    response = json.loads(r.content)
    
    df = pd.DataFrame({
        'Month:': [_[0] for _ in response['time_intervals']],
        'Users': response['data'][0]['metrics'][0],
        'Visits': response['data'][0]['metrics'][1],
        'Views': response['data'][0]['metrics'][2],
        'Глубина': response['data'][0]['metrics'][3],
        "Время на сайте": [i/86400 for i in response['data'][0]['metrics'][4]], # Метрика отдает время в секундах. Деление на 86400 для быстрого форматирования в эксель
        "Отказы до 15 сек": [i/100 for i in response['data'][0]['metrics'][5]]
                      })
    return df

# 2 запрос. Доля сессий с глубиной 1 стр.
# Функция возвращает pd.Series с кол-вом сессий с глубиной = 1 страница

def query2(counter, region):
    exclude_zen = f"ym:pv:URLDomain!@'zen.yandex' AND ym:s:startURL=@'{counter[1]}'"
    region_fltr = f"ym:s:regionAreaIsoName=='{region}'"
    payload = {
        'metrics': 'ym:s:visits',
        'accuracy': '1',
        'attribution': 'last',
        'date1': '2015-12-01',
        'date2': '2021-03-31',
        'group': 'month',
        'filters': f"ym:s:pageViews==1 AND {region_fltr} AND {exclude_zen}",
#         'filters': f"ym:s:regionAreaIsoName=='{region}' AND ym:s:pageViews==1",
        'ids': counter[0]
        }

    r = requests.get('https://api-metrika.yandex.ru/stat/v1/data/bytime', params=payload, headers=header)
    response = json.loads(r.content)
    return pd.Series(response['data'][0]['metrics'][0])

for counter in ids:
    print (counter['name'], counter['id'])
    site = counter['name']
    df = pd.DataFrame
    df = query1([counter['id'], site], counter['region'])
    df['Доля сессий с гл.1 стр.'] = query2([counter['id'], site], counter['region'])/df['Visits']
    
    df.to_excel(f'4_1+2_{site}.xlsx')

In [ ]:
# 3 запрос. Получает структуру визитов по источникам
# Отдает ДАТАФРЕЙМ с 4 столбцами: Прямые заходы, Рефералы, Поиск, Соцсети

def query3(counter, region):
    exclude_zen = f"ym:pv:URLDomain!@'zen.yandex' AND ym:s:startURL=@'{counter[1]}'"
    region_fltr = f"ym:s:regionAreaIsoName=='{region}'"
    payload = {
        'metrics': 'ym:s:visits',
        'accuracy': '1',
        'attribution': 'last',
        'date1': '2015-12-01',
        'date2': '2021-03-31',
        'group': 'month',
        'dimensions': "ym:s:LastTrafficSource",
        'filters': f"{region_fltr} AND {exclude_zen}",
#         'filters': f"ym:s:regionAreaIsoName=='{region}'",
        'ids': counter[0]
        }

    r = requests.get('https://api-metrika.yandex.ru/stat/v1/data/bytime', params=payload, headers=header)
    response = json.loads(r.content)
    
    df = pd.DataFrame()
    for i in response['data']:
        df[i['dimensions'][0]['name']] = i['metrics'][0]
    df['Рефералы'] = df['Переходы из рекомендательных систем'] + df['Переходы по ссылкам на сайтах']
    df = df[['Прямые заходы', 'Рефералы', 'Переходы из поисковых систем', 'Переходы из социальных сетей']]    
    
    
    return df

for counter in ids:
    print (counter['name'], counter['id'])
    site = counter['name']
    data = query3([counter['id'], site], counter['region'],)
    
    data.to_excel(f'4_3_{site}.xlsx')

In [ ]:
# 4 запрос. Трафик Яндекс.Новостей, Дзен, Гугл Дискавери
def query4(counter, region):
    
    exclude_zen = f"ym:pv:URLDomain!@'zen.yandex' AND ym:s:startURL=@'{counter[1]}'"
    region_fltr = f"ym:s:regionAreaIsoName=='{region}'"
    
    channels = {
        'Яндекс Новости': "ym:s:lastReferalSource=@'yandex' AND ym:s:lastReferalSource!@'zen'",
        'Zen.yandex': "ym:s:lastRecommendationSystemFavicon=='Яндекс Дзен' OR ym:s:lastReferalSource=@'zen.'",
        'Google Discover': "ym:s:lastRecommendationSystemFavicon=='Google Discover' OR ym:s:lastReferalSource=@'googleapis' OR ym:s:lastReferalSource=@'android-app'"        
    }
    
    df = pd.DataFrame() #(columns=['Яндекс Новости', 'Zen.yandex', 'Google Discover'])
    
    for channel, fltr in channels.items():
        payload = {
            'metrics': 'ym:s:visits',
            'accuracy': '1',
            'attribution': 'last',
            'date1': '2015-12-01',
            'date2': '2021-03-31',
            'group': 'month',
            'filters': f"{region_fltr} AND ({fltr}) AND {exclude_zen}",
#             'filters': f"ym:s:regionAreaIsoName=='{region}' AND {fltr}",
            'ids': counter[0]
            }
        r = requests.get('https://api-metrika.yandex.ru/stat/v1/data/bytime', params=payload, headers=header)
        response = json.loads(r.content)
        df[channel] = response['data'][0]['metrics'][0]
    
    return df

for counter in ids:
    print (counter['name'], counter['id'], counter['region'])
    site = counter['name']
    data = query4([counter['id'],site], counter['region'])
    
    data.to_excel(f'4_4_{site}.xlsx')